In [4]:
import numpy as np 
import nltk
from nltk.corpus import inaugural
from nltk.tokenize import word_tokenize
nltk.download('brown')
nltk.download('punkt')
nltk.download('universal_tagset')
from nltk.corpus import brown

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [18]:
## Data Processing
tagged_sentences = brown.tagged_sents(tagset='universal')
num_sentences = len(brown.tagged_sents())

START_SYMBOL = '^'
END_SYMBOL = '.'

sentences = []
for i in range(num_sentences):
    sentences.append(tagged_sentences[i])
    
for i in range(num_sentences):
    for j in range(len(sentences[i])):
        x = list(sentences[i][j])
        x[0] = x[0].lower()
        sentences[i].pop(j)
        sentences[i].insert(j, tuple(x))
        
for i in range(num_sentences):
    sentences[i] = [(START_SYMBOL, START_SYMBOL)] + sentences[i]

[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')]


In [19]:
## Transition Probabilities
count_tags_bigram = {}
for j in range(len(sentences)):
    for i, item in enumerate(sentences[j]):
        if(i == len(sentences[j])-1):
            continue
        
        tuple_tags = (sentences[j][i+1][1], sentences[j][i][1])
        if tuple_tags in count_tags_bigram.keys():
            count_tags_bigram[tuple_tags] += 1
            
        else:
            count_tags_bigram[tuple_tags] = 1
            
probability_tags_bigram = {}
total = 0
for key in count_tags_bigram.keys():
    total += count_tags_bigram[key]
    
    
for key in count_tags_bigram.keys():
    probability_tags_bigram[key] = count_tags_bigram[key]/total

In [20]:
## Lexical Independence Probabilities / Emission Probability
emission_probabilities_dict = {'.' : {'.' : 1}}

for j in range(len(sentences)):
    for i, item in enumerate(sentences[j]):
        if(i == len(sentences[j])-1):
            continue
        
        if(sentences[j][i][1] in emission_probabilities_dict):
            if(sentences[j][i][0] in emission_probabilities_dict[sentences[j][i][1]]):
                emission_probabilities_dict[sentences[j][i][1]][sentences[j][i][0]] += 1
                
            else:
                emission_probabilities_dict[sentences[j][i][1]][sentences[j][i][0]] = 1
                
        else:
            emission_probabilities_dict[sentences[j][i][1]] = {}
            emission_probabilities_dict[sentences[j][i][1]][sentences[j][i][0]] = 1
            
tags = list(emission_probabilities_dict.keys())
for key1 in emission_probabilities_dict.keys():
    total = 0
    for key2 in emission_probabilities_dict[key1].keys():
        total += emission_probabilities_dict[key1][key2]
    
    for key3 in emission_probabilities_dict[key1].keys():
        emission_probabilities_dict[key1][key3] /= total

In [21]:
input_sent = "reading is an essential skill."
input_sent_tokenized = word_tokenize(input_sent)

input_sent_tokenized

['reading', 'is', 'an', 'essential', 'skill', '.']

In [22]:
viterbi_matrix = []
ls = []
for tag in tags:
    if((tag, '^') in probability_tags_bigram.keys()):
        if(input_sent_tokenized[0] in emission_probabilities_dict[tag].keys()):
            ls.append(('^', probability_tags_bigram[(tag, '^')] * emission_probabilities_dict[tag][input_sent_tokenized[0]]))
        
        else:
            ls.append(('^', 0))
    
    else:
        ls.append(('^', 0))
        
viterbi_matrix.append(ls)

tags = list(tags)
for i in range(1, len(input_sent_tokenized)):
    ls = [] 
    for tag in tags:
        max_probability = 0
        max_tag = '$Can$'
        for j in range(len(viterbi_matrix[i-1])):
            if(viterbi_matrix[i-1][j][1] > 0):
                prev_tag = tags[j]
                prev_probability = viterbi_matrix[i-1][j][1]
                
                if (tag, prev_tag) in probability_tags_bigram.keys() and input_sent_tokenized[i] in emission_probabilities_dict[tag].keys():
                    total_probability  = prev_probability * probability_tags_bigram[(tag, prev_tag)] * emission_probabilities_dict[tag][input_sent_tokenized[i]]
                    if(total_probability > max_probability):
                        max_probability = total_probability
                        max_tag = prev_tag            
        
        ls.append((max_tag, max_probability))                
    viterbi_matrix.append(ls)  

In [23]:
for i,item in enumerate(viterbi_matrix):
    max_val = -1
    max_tag = ''
    for x in item:
        if(x[1] > max_val):
            max_val = x[1]
            max_tag = x[0]
    if(i == 0): print('Start: ' + max_tag)
    else: print(input_sent_tokenized[i-1] + ": " + max_tag)


Start: ^
reading: NOUN
is: VERB
an: DET
essential: ADJ
skill: NOUN
